# Mansa loans

## EDA

We start by performing an EDA of the datasets.

In [1]:
import pandas as pd
from src.data_tools import get_data

In [2]:
# read the csv files
df_accounts = get_data("data/accounts.csv")
df_transactions = get_data("data/transactions.csv")

In [3]:
df_accounts.head()

,id,balance,update_date
0,0,13.63,2021-07-02
1,1,12.91,2021-07-02
2,2,19.84,2021-07-02
3,3,130.00,2021-07-02
4,4,2806.75,2021-07-02


In [4]:
df_transactions.head()

,account_id,date,amount
0,0,2020-10-16,200.0
1,0,2020-10-16,-192.0
2,0,2020-10-16,200.0
3,0,2020-10-16,-24.0
4,0,2020-10-16,-50.0


#### Checking for null values

In [5]:
df_accounts.isna().any()

id             False
balance        False
update_date    False
dtype: bool

In [6]:
df_transactions.isna().any()

account_id    False
date          False
amount        False
dtype: bool

There is no null value in any of the two datasets.


#### Checking for duplicates

In [7]:
df_accounts.duplicated().any()

False

In [8]:
df_transactions.duplicated().any()

True

There are duplicates in the transactions. Lets look at some of them

In [9]:
n_duplicated = df_transactions.duplicated().sum()
dup_percent = int(n_duplicated / df_transactions.shape[0]*100)
print(f'there are {n_duplicated} duplicates in transactions.csv, representing {dup_percent}% of the data.')

there are 28661 duplicates in transactions.csv, representing 5% of the data.


In [10]:
df_dup = df_transactions[df_transactions.duplicated()]
df_dup.head()

,account_id,date,amount
2,0,2020-10-16,200.0
11,0,2020-10-17,0.0
14,0,2020-10-19,0.0
23,0,2020-10-21,0.0
45,0,2020-10-29,-5.5


In [11]:
df_transactions[
    (df_transactions.account_id == 0) 
    & (df_transactions.date == '2020-10-16') 
    &(df_transactions.amount == 200.0)]

,account_id,date,amount
0,0,2020-10-16,200.0
2,0,2020-10-16,200.0


It is difficult, without further information to know if the above transactions are legitimate similar transactions or duplicates coming from an error when the data were processed. We decide to keep these duplicates. It would be interesting to enrich the transactions data with a transaction id. We could decide with certainty with such an id if similar transactions on a given account and a given date are duplicates or legitimates.

#### Account history distribution

Let's check the history distribution of the different accounts in our data.

In [12]:
# we change the date column from str to datetime
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
group = df_transactions.groupby('account_id')['date']

In [13]:
history_df = group.agg(["min", "max"])
history = history_df['max'] - history_df['min']
history.head()

account_id
0   257 days
1   231 days
2   100 days
3   176 days
4   293 days
dtype: timedelta64[ns]

In [14]:
trigger = pd.Timedelta(180, "d")
n_accounts = (history > trigger).sum()
frac_long_account = int(n_accounts / len(history) * 100)
print(f'{n_accounts} accounts, which represent {frac_long_account}% of all the accounts, have an history of more than 6 months')

823 accounts, which represent 65% of all the accounts, have an history of more than 6 months


We want to build a model that make predictions using 6 months of data so we'll focus on accounts with more than 6 months history. We use get_df_with_history to select these accounts.

In [15]:
from src.data_tools import get_df_with_history
df_accounts, df_transactions = get_df_with_history(df_accounts, df_transactions)

In [16]:
n_transac = len(set(df_transactions['account_id']))
print(f'We are left with {len(df_accounts)} accounts in df_accounts and {n_transac} in df_transactions which is as expected.')

We are left with 860 accounts in df_accounts and 860 in df_transactions which is as expected.


## Processing the data for training

We are looking to predict the next month outgoing given the last 6 month of transaction. For that purpose it make sense to divide the history of transactions in 30 days buckets and to calculate the total amount of inflow and outflow for each 30 days bucket. 

When processing the data, we keep the last 2 months for testing and the previous 2 months for validation so that we are sure there is no data leakage when training and testing. 
 
For some accounts there are more than 11 months of data. In this case we'll get from the account several training 
samples. For example if an account has 12 month of data, the last 4 months are used for validation and testing, so we 
are left with 8 months of data for training. We'll have then 2 training samples from this account (2 rows in X_train):
  - one sample with the first 6 months for predicting the 7 month
  - A second sample with the 2nd month to the 7th month for predicting the 8th month.

In [17]:
from src.data_tools import get_training_data
# When processing the data, we keep the last 2 months for testing and the previous 2 months for validation
training_data = get_training_data(test_size=2)

In [18]:
training_data.keys()

dict_keys(['train', 'val', 'test'])

In [19]:
training_data['train'].keys()

dict_keys(['X', 'y'])

The data are split between training, validation and test. Each of these split is a dictionary with the input of the model: 'X' and the target variable: 'y'. Let's check the relative proportion of these different splits:

In [20]:
n_train = training_data['train']['X'].shape[0]
n_val = training_data['val']['X'].shape[0]
n_test = training_data['test']['X'].shape[0]
n = n_train + n_val + n_test
print(f'There is a total of {n} data')
print(f'{n_train/n*100:.1f}% for training, {n_val/n*100:.1f}% for validation and {n_test/n*100:.1f}% for testing')

There is a total of 13520 data
75.5% for training, 12.3% for validation and 12.2% for testing


This is an acceptable split of the data for training our model. Lets look at how the data were processed:

In [21]:
training_data['train']['X'].head()

,1M inflow,2M inflow,3M inflow,4M inflow,5M inflow,6M inflow,1M outflow,2M outflow,3M outflow,4M outflow,5M outflow,6M outflow,initial_balance
0,90.00,0.00,0.00,50.00,0.00,0.00,0.00,-50.00,-50.00,-100.00,-50.00,-50.00,2525.00
1,4138.47,5376.46,3833.00,3600.00,3097.20,2000.00,-4946.87,-4929.89,-3876.41,-4195.21,-2071.26,-1680.31,70.20
2,0.00,0.00,0.00,0.00,1182.00,4034.00,0.00,0.00,0.00,0.00,-21.27,-2088.17,-80.94
3,1705.00,646.00,231.85,312.35,455.07,4787.98,-1351.20,-1039.65,-217.80,-255.07,-271.26,-754.99,280.66
4,0.00,0.00,0.00,0.00,1040.00,714.00,0.00,0.00,0.00,0.00,0.00,-68.65,123.69


These data represent 6 months of transaction history. For example, '1M inflow' gives the total amount of positive transactions in the accounts during the first month of the history considered and '6M outflow' gives the total amount of negative transaction in the accounts during the last month of the history considered. initial_balance give the balances of the account at the beginning of the 6 months period considered.

We have organised the data as needed for the next step:

In [22]:
X_train = training_data['train']['X']
y_train = training_data['train']['y']
X_val = training_data['val']['X']
y_val = training_data['val']['y']
X_test = training_data['test']['X']
y_test = training_data['test']['y']


## Model choice

Given that time is limited we'll compare different model performance with their default setting to choose our model

In [23]:
# we perform some scaling on the data. It is not necessary for tree based models but we'll keep a common 
# groundwork for our analysis
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


### Regression

Regression will give us a benchmark. It's unlikely to be the most performant model, we should be able to do better with ensemble models.

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# we start with linear regression
reg = LinearRegression()
reg.fit(X_train, y_train)
y_predict = reg.predict(X_val)
r2 = r2_score(y_val, y_predict)
print(f'Linear regression regression r2: {r2}')

Linear regression regression r2: 0.26590342126231004


In [25]:
from sklearn.linear_model import Ridge

# we impose a penalty using Ridge regression
ridge = Ridge()
ridge.fit(X_train, y_train)
y_predict = ridge.predict(X_val)
r2 = r2_score(y_val, y_predict)
print(f'Ridge regression r2: {r2}')

Ridge regression r2: 0.26592745106054116


In [26]:
from sklearn.linear_model import Lasso

# we now impose a penalty using Lasso regression
lasso = Lasso()
lasso.fit(X_train, y_train)
y_predict = lasso.predict(X_val)
r2 = r2_score(y_val, y_predict)
print(f'Lasso regression r2: {r2}')

Lasso regression r2: 0.2658387256598487


### Random Forests

In [27]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=48)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_val)
r2 = r2_score(y_val, y_predict)
print(f'Random forest r2: {r2}')

Random forest r2: 0.1799939470557319


### Light GBM

In [28]:
from lightgbm import LGBMRegressor
reg_lg = LGBMRegressor(n_estimators=100, random_state=48)
reg_lg.fit(X_train, y_train)
y_predict = reg_lg.predict(X_val)
r2 = r2_score(y_val, y_predict)
print(f'Light gbm r2: {r2}')

Light gbm r2: 0.28423212148743804


Light GBM seems the most promising model. We'll use Light GBM from now on. We don't need to scale the data for tree based models so we adjust accordingly

In [29]:
X_train = training_data['train']['X']
X_val = training_data['val']['X']

### Hyperparameter tuning for LightGBM

We use Optuna for hyperparameters tuning:
- parameters that control the tree structure: num_leaves and max_depth (between 3 and 12). LGBM documentation indicates that num_leaves should be < 2^(max_depth)
- parameters for better accuracy: n_estimators and learning_rate 
- parameters to control overfitting: bagging fraction
- some other parameters


In [30]:
import optuna
from src.hyper_params import get_objective

/Users/gdegournay/opt/anaconda3/envs/mansa/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
study = optuna.create_study(direction='minimize')
objective = get_objective(X_train, X_val, y_train, y_val)
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-04-24 08:08:33,169] A new study created in memory with name: no-name-156652e0-be1c-4cf4-976a-81255775dbc1
[I 2022-04-24 08:08:33,228] Trial 0 finished with value: 7729.432309399014 and parameters: {'n_estimators': 100, 'reg_alpha': 5.4547207424579955, 'reg_lambda': 0.006695853706927359, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 2, 'num_leaves': 50, 'min_child_samples': 219}. Best is trial 0 with value: 7729.432309399014.


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7729.43
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 7729.43
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 8069.26
[200]	valid_0's rmse: 7464.32
[300]	valid_0's rmse: 7355.88


[I 2022-04-24 08:08:33,444] Trial 1 finished with value: 7349.144280399479 and parameters: {'n_estimators': 1250, 'reg_alpha': 0.02336708435466059, 'reg_lambda': 0.0034719329416884066, 'colsample_bytree': 0.5, 'subsample': 0.6, 'learning_rate': 0.006, 'max_depth': 5, 'num_leaves': 82, 'min_child_samples': 100}. Best is trial 1 with value: 7349.144280399479.
[I 2022-04-24 08:08:33,506] Trial 2 finished with value: 7448.456518798932 and parameters: {'n_estimators': 2000, 'reg_alpha': 0.01466992340477327, 'reg_lambda': 0.022722547964770613, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 0.02, 'max_depth': 2, 'num_leaves': 92, 'min_child_samples': 107}. Best is trial 1 with value: 7349.144280399479.
[I 2022-04-24 08:08:33,557] Trial 3 finished with value: 8173.121825378745 and parameters: {'n_estimators': 100, 'reg_alpha': 0.010565164519354448, 'reg_lambda': 1.4835949734666976, 'colsample_bytree': 0.3, 'subsample': 0.4, 'learning_rate': 0.006, 'max_depth': 3, 'num_leaves': 87,

[400]	valid_0's rmse: 7367.41
Early stopping, best iteration is:
[344]	valid_0's rmse: 7349.14
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7460.43
[200]	valid_0's rmse: 7524.91
Early stopping, best iteration is:
[118]	valid_0's rmse: 7448.46
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 8173.12
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 8173.12
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7495.72
[200]	valid_0's rmse: 7504.87


[I 2022-04-24 08:08:33,643] Trial 4 finished with value: 7477.373364611033 and parameters: {'n_estimators': 1250, 'reg_alpha': 0.001951693968452124, 'reg_lambda': 0.5789276147325875, 'colsample_bytree': 0.7, 'subsample': 0.7, 'learning_rate': 0.02, 'max_depth': 3, 'num_leaves': 56, 'min_child_samples': 184}. Best is trial 1 with value: 7349.144280399479.
[I 2022-04-24 08:08:33,705] Trial 5 finished with value: 7578.339480793068 and parameters: {'n_estimators': 250, 'reg_alpha': 5.6837038890475275, 'reg_lambda': 0.1796889699124674, 'colsample_bytree': 0.3, 'subsample': 1.0, 'learning_rate': 0.017, 'max_depth': 2, 'num_leaves': 48, 'min_child_samples': 227}. Best is trial 1 with value: 7349.144280399479.
[I 2022-04-24 08:08:33,789] Trial 6 finished with value: 7444.223233450637 and parameters: {'n_estimators': 1250, 'reg_alpha': 0.004200953536753077, 'reg_lambda': 0.0037161481573136975, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.01, 'max_depth': 2, 'num_leaves': 84, 'm

...

Early stopping, best iteration is:
[221]	valid_0's rmse: 7301.86
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7609.82
[200]	valid_0's rmse: 7342.37
[300]	valid_0's rmse: 7372.52
Early stopping, best iteration is:
[218]	valid_0's rmse: 7339.24
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7681.6
[200]	valid_0's rmse: 7403.7


[I 2022-04-24 08:08:42,593] Trial 46 finished with value: 7393.837484248751 and parameters: {'n_estimators': 250, 'reg_alpha': 0.004956210962547387, 'reg_lambda': 3.539695177530633, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 10, 'num_leaves': 75, 'min_child_samples': 149}. Best is trial 23 with value: 7295.7965560751.
[I 2022-04-24 08:08:42,681] Trial 47 finished with value: 7777.019276656346 and parameters: {'n_estimators': 100, 'reg_alpha': 0.002025152148772397, 'reg_lambda': 1.9254707128909907, 'colsample_bytree': 0.4, 'subsample': 0.7, 'learning_rate': 0.01, 'max_depth': 2, 'num_leaves': 100, 'min_child_samples': 93}. Best is trial 23 with value: 7295.7965560751.


Did not meet early stopping. Best iteration is:
[228]	valid_0's rmse: 7393.84
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7777.02
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 7777.02
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7744.85
[200]	valid_0's rmse: 7477.71


[I 2022-04-24 08:08:43,178] Trial 48 finished with value: 7475.498985933856 and parameters: {'n_estimators': 750, 'reg_alpha': 1.49056651431573, 'reg_lambda': 0.44324882420774714, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.01, 'max_depth': 10, 'num_leaves': 49, 'min_child_samples': 192}. Best is trial 23 with value: 7295.7965560751.
[I 2022-04-24 08:08:43,342] Trial 49 finished with value: 7327.090551163841 and parameters: {'n_estimators': 2000, 'reg_alpha': 4.288200799443621, 'reg_lambda': 0.20097179526944384, 'colsample_bytree': 0.3, 'subsample': 0.5, 'learning_rate': 0.02, 'max_depth': 5, 'num_leaves': 34, 'min_child_samples': 129}. Best is trial 23 with value: 7295.7965560751.


[300]	valid_0's rmse: 7493.71
Early stopping, best iteration is:
[216]	valid_0's rmse: 7475.5
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 7330.3
[200]	valid_0's rmse: 7446.07
Early stopping, best iteration is:
[103]	valid_0's rmse: 7327.09
Number of finished trials: 50
Best trial: {'n_estimators': 2000, 'reg_alpha': 2.1932499270244104, 'reg_lambda': 3.1834449128431275, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 10, 'num_leaves': 39, 'min_child_samples': 116}


Let's create an LGBMRegresor with the best hyperparameters found in the previous study

In [32]:
params=study.best_params   
params['random_state'] = 48
params['metric'] = 'rmse'
params

{'n_estimators': 2000,
 'reg_alpha': 2.1932499270244104,
 'reg_lambda': 3.1834449128431275,
 'colsample_bytree': 0.4,
 'subsample': 0.8,
 'learning_rate': 0.014,
 'max_depth': 10,
 'num_leaves': 39,
 'min_child_samples': 116,
 'random_state': 48,
 'metric': 'rmse'}

In [34]:
model = LGBMRegressor(**params)
model.fit(X_train,y_train)
r2 = model.score(X_val, y_val)


In [35]:
r2 = model.score(X_val, y_val)
print(f'Light gbm r2 after hyper parameters tuning: {r2}')

Light gbm r2 after hyper parameters tuning: 0.34828934506153364


Fine tuning the hyperparameters of the model has greatly improved it r2, from 0.28 to 0.35. We save so model so that it can be used by the API and move to the test part

### Tests

In [36]:
from src.model import load_model
from src.train_model import eval_model

In [37]:
model = load_model('lgb')

In [39]:
result = eval_model(model)
result

{'r2': 0.33630406990008166, 'rmse': 11606.626269252638, 'rmse_percent': 188.3}

The test results are close to the validation results, which is positive. On the other hand the rmse is still too large at 188% of the average target variable value.